## Importing basic libraries

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

## Importing Data

In [2]:
train_ds,val_ds,test_ds=tfds.load('imdb_reviews',split=['train','test[:50%]','test[50%:]'],as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteIGQI4W/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteIGQI4W/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteIGQI4W/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
for review,label in train_ds.shuffle(buffer_size=15).take(5):
  print(review.numpy())
  print(label.numpy())

b"This is a film which should be seen by anybody interested in, effected by, or suffering from an eating disorder. It is an amazingly accurate and sensitive portrayal of bulimia in a teenage girl, its causes and its symptoms. The girl is played by one of the most brilliant young actresses working in cinema today, Alison Lohman, who was later so spectacular in 'Where the Truth Lies'. I would recommend that this film be shown in all schools, as you will never see a better on this subject. Alison Lohman is absolutely outstanding, and one marvels at her ability to convey the anguish of a girl suffering from this compulsive disorder. If barometers tell us the air pressure, Alison Lohman tells us the emotional pressure with the same degree of accuracy. Her emotional range is so precise, each scene could be measured microscopically for its gradations of trauma, on a scale of rising hysteria and desperation which reaches unbearable intensity. Mare Winningham is the perfect choice to play her m

## Text Standardization

In [27]:
def standardization(inp_text):
  no_html=tf.strings.regex_replace(inp_text,"<[^>]+>","") # removing html tags
  no_spl_char=tf.strings.regex_replace(no_html,"[^a-zA-Z0-9\s]","") # removing special chars,punctuations
  no_upper=tf.strings.lower(no_spl_char) # lower case
  return no_upper



In [28]:
# getting vocabularies
vectors=tf.keras.layers.TextVectorization(
    standardize=standardization,
    max_tokens=10000,
    output_mode='int',
    output_sequence_length=200
)

In [29]:
training_data=train_ds.map(lambda x,y:x)

In [30]:
vectors.adapt(training_data)

In [33]:
len(vectors.get_vocabulary())

10000

In [35]:
# getting the review and label---> converting it into respective vector---> returning it along with label
def vectorization(review,label):
  return vectors(review),label

In [37]:
training_reviews=train_ds.map(vectorization)
validation_reviews=val_ds.map(vectorization)

In [41]:
for review,label in training_reviews.take(1):
  print(review)

tf.Tensor(
[  10   13   33  411  384   17   89   26    1    8   32 1337 3522   40
  491    1  192   22   84  149   18   10  215  317   26   64  239  212
    8  483   54   64   84  111   95   21 5502   10   91  636  736   10
   17    7   33  393 9553  169 2442  406    2   87 1204  135   65  142
   52    2    1 7407   65  245   64 2832   16    1 2850    1    1 1415
 4971    3   39    1 1561   15 3522   13  156   18    4 1204  880 7872
    8    4   17   12   13 4036    5   98  145 1232   11  236  695   12
   48   22   91   37   10 7285  149   37 1337    1   49  396   11   95
 1147  840  140    9    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0 

In [42]:
train_dataset=training_reviews.batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset=validation_reviews.batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)